<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [50]:
# Doing some of the imports here
import pandas as pd
import tensorflow as tf
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [2]:
PATH = os.path.join(os.path.curdir, "WA_Fn-UseC_-Telco-Customer-Churn+(1).csv")
PATH

'.\\WA_Fn-UseC_-Telco-Customer-Churn+(1).csv'

In [22]:
# Loading the data set
df = pd.read_csv(PATH)
df.shape

(7043, 21)

In [4]:
# Looking for null values
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [5]:
# This means that we are doing a binary classification for this data
df["Churn"].unique()

array(['No', 'Yes'], dtype=object)

In [6]:
df.head(2)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No


In [7]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [8]:
for col in df.columns.tolist():
    print(f"vals for {col}")
    print(pd.unique(df[col]))
    print("----------------")


vals for customerID
['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']
----------------
vals for gender
['Female' 'Male']
----------------
vals for SeniorCitizen
[0 1]
----------------
vals for Partner
['Yes' 'No']
----------------
vals for Dependents
['No' 'Yes']
----------------
vals for tenure
[ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39]
----------------
vals for PhoneService
['No' 'Yes']
----------------
vals for MultipleLines
['No phone service' 'No' 'Yes']
----------------
vals for InternetService
['DSL' 'Fiber optic' 'No']
----------------
vals for OnlineSecurity
['No' 'Yes' 'No internet service']
----------------
vals for OnlineBackup
['Yes' 'No' 'No internet service']
----------------
vals for DeviceProtection
['No' 'Yes' 'No internet service']
----------------
vals f

In [33]:
# Will be doing some one hot encoding of the values that are strings
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
import category_encoders as ce
from sklearn.preprocessing import MinMaxScaler

In [10]:
# trying the drop of the dataframe
d = df.copy()

In [40]:
y = df["Churn"]

In [41]:
x_features = [x for x in df.columns.tolist() if x != "Churn"]

In [42]:
x = df[x_features]
x.head(1)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85


In [51]:
# Doing a train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=.85, random_state=42,
stratify=y)

In [52]:
# printing out the shapes
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(5986, 20) (5986,) (1057, 20) (1057,)


In [47]:
def prep_data(df):
    the_df = df.copy() # copying the data 
     
    # doing some droping of the id
    the_df = the_df.drop(columns=["customerID"], axis=1)

    # now doing going to change the TotalCharges to change it to 
    # a numeric
    the_df["TotalCharges"] = pd.to_numeric(the_df["TotalCharges"], errors="coerce")

    # Will now do dropping of the rows that have null in them in the TotalCharges
    the_df = the_df.dropna()
    # doing some ordinal encoding
    # ord = OrdinalEncoder(categories=["gender", "Partner", "Dependents"])
    # create the one hot encoding
    vals_to_encode = ["gender", "Partner", "Dependents", "PhoneService", 
                    "MultipleLines", "InternetService", "OnlineSecurity", "OnlineBackup", 
                    "DeviceProtection", "TechSupport", "StreamingTV", 
                    "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod"]
    encoder = ce.OneHotEncoder(use_cat_names=True, cols=vals_to_encode)
    
    encoded_data = encoder.fit_transform(the_df)

    # using a scaler to scale the data
    minScaler = MinMaxScaler()
    scaled_data = minScaler.fit_transform(encoded_data)
    
    frame= pd.DataFrame(data=scaled_data, columns=encoder.get_feature_names())
    return scaled_data, frame

In [53]:
x_train, x_train_df = prep_data(x_train)
x_test, x_test_df = prep_data(x_test)

In [ ]:
# Doing some imports for the model and tuning
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [55]:
# Looking at the baseLIne
y_train.value_counts(normalize=True)

No     0.734547
Yes    0.265453
Name: Churn, dtype: float64

In [56]:
y_test.value_counts(normalize=True)

No     0.735099
Yes    0.264901
Name: Churn, dtype: float64

In [58]:
# Doing more imports
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
# Creating the model



## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?